# setup compute

In [ ]:
!pip install prefect[dask] dask-jobqueue

In [ ]:
#pip install prefect[dask] dask-jobqueue
# prefect config set PREFECT_API_URL="https://ard-modeling-service.slac.stanford.edu/api" 

from dask.distributed import Client
from dask_jobqueue import SLURMCluster

# Simplify SLURMCluster parameters to basic working configuration
cluster_kwargs = {
    "cores": 12,  
    "memory": "10GB",  # Correct spacing
    "shebang": "#!/bin/bash",
    "account": "facet",
    "walltime": "00:10:00",
    "job_script_prologue": ["source ~/.bashrc"],
    # Only basic directives, add more as needed and test each step
    "job_extra_directives": ["-q debug", "--partition=milano", "--qos=preemptable"],
}




In [ ]:
cluster = SLURMCluster(**cluster_kwargs)
print(cluster.job_script())


In [ ]:
slurm_jobs = 1
cluster.scale(jobs=slurm_jobs)
client = Client(cluster)

# define code

In [ ]:
%time
@delayed
def slow_square(x):
    time.sleep(1)  # Simulate a delay
    return x * x

# Generate tasks using a for loop
tasks = [slow_square(i) for i in range(10)]

# Compute the results in parallel
results = compute(*tasks)

print("Results:", results)